# Data Preparation

### Imports

In [ ]:
#libraries
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

#custom
import utils
import plots
import transformers as tran

LABEL = "1Y_default"
SEED = 42

%matplotlib inline  
%load_ext autoreload
%autoreload 2

pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

## Load data

In [ ]:
#load data from csv
data_full = pd.read_csv("LoanData.csv")

print("Raw data shape: {}".format(data_full.shape))

###  Preselect variables

In [ ]:
#select only relevant variables available at application
to_keep = ["LoanDate","DefaultDate", "Age", "ProbabilityOfDefault", "Amount", "AmountOfPreviousLoansBeforeLoan", "AppliedAmount", "City", "Country", "County", "DebtToIncome",
           "Education", "EmploymentDurationCurrentEmployer", "EmploymentPosition", "EmploymentStatus",
           "ExistingLiabilities", "FreeCash", "Gender", "HomeOwnershipType", "IncomeFromChildSupport", "IncomeFromFamilyAllowance",
           "IncomeFromLeavePay", "IncomeFromPension", "IncomeFromPrincipalEmployer", "IncomeFromSocialWelfare", 
           "IncomeOther", "IncomeTotal", "Interest", "LanguageCode", "MaritalStatus", "MonthlyPayment", "NewCreditCustomer",
           "NoOfPreviousLoansBeforeLoan", "NrOfDependants", "OccupationArea", "UseOfLoan", "VerificationType", "WorkExperience"   
]

data_df = data_full[to_keep]
print("Data shape: {}".format(data_df.shape))

### Create label

In [ ]:
#create label (1Y default flag) based on default day
#convert to date
data_df["DefaultDate"] =pd.to_datetime(data_df["DefaultDate"], yearfirst=True)
data_df["LoanDate"] = pd.to_datetime(data_df["LoanDate"], yearfirst=True)

#days between start and default
data_df["default_delta"] = (data_df["DefaultDate"].values - data_df["LoanDate"].values)

def date_to_int(date):
    try:
        return date.days
    except:
        return np.nan

data_df["default_delta"] = data_df["default_delta"].apply(date_to_int)
#default flag
data_df["1Y_default"] = (data_df["default_delta"].values <= 365)*1

### Filter rows

In [ ]:
#select only loans originated during years 2013-2017
data_df["OriginYear"] = data_df["LoanDate"].apply(lambda date: date.year)
#select only loans where 1Y default is known
data_df = data_df[data_df["LoanDate"] < pd.Timestamp('2017-06-01 00:00:00')]

In [ ]:
plots.dependence_cat(data_df[LABEL], data_df["OriginYear"], estimator=np.mean, y_label="Average deafult rate")

In [ ]:
plots.dependence_cat(data_df["ProbabilityOfDefault"], data_df["OriginYear"], estimator=np.mean, y_label="Average predicted PD")

In [ ]:
#select only years 2014+
data_df = data_df[data_df["LoanDate"] < pd.Timestamp('2017-06-01 00:00:00')]

#drop unwanted columns
data_filtered_df = data_df.drop(["LoanDate", "DefaultDate", "ProbabilityOfDefault", "default_delta", "OriginYear"], axis=1)
print("Filtered data shape: {}".format(data_filtered_df.shape))

## Feature engineering

### Drop irrelevant features

In [ ]:
#drop City, County, EmploymentPosition: too high cardinality with only few observations
data_filtered_df = data_filtered_df.drop(["City", "County", "EmploymentPosition"], axis=1)

### Check data types

In [ ]:
#some features has missing values as well as special category (0 or -1) for missing
to_correct = ["Education", "MaritalStatus", "VerificationType"]
for feature in to_correct:
    data_filtered_df[feature].replace(0, np.nan, inplace=True)

data_filtered_df["OccupationArea"].replace(-1, np.nan, inplace=True)

In [ ]:
#check for dtypes
data_filtered_df.dtypes

In [ ]:
#correct dtypes:
#correct NrOfDependants
def correct_dependants(row):
    try:
        return int(row)
    except:
        return row
    
data_filtered_df["NrOfDependants"] = data_filtered_df["NrOfDependants"].apply(correct_dependants)

#numerically encoded categoricals
to_categorical = ["Education", "EmploymentStatus", "Gender", "HomeOwnershipType", "LanguageCode", "MaritalStatus",
                 "NewCreditCustomer", "OccupationArea", "UseOfLoan", "VerificationType", "Country", "NrOfDependants", 
                  "NrOfDependants", "WorkExperience", "EmploymentDurationCurrentEmployer"]
for feature in to_categorical:
    data_filtered_df[feature] = data_filtered_df[feature].astype("category")

In [ ]:
data_filtered_df.dtypes

### Missing values

In [ ]:
#check missings
utils.check_missing(data_filtered_df)

In [ ]:
#discretize and create new category for missing
data_filtered_df["MonthlyPayment"] = data_filtered_df["MonthlyPayment"].apply(utils.make_bins, tresholds=[100])
data_filtered_df["MonthlyPayment"] = data_filtered_df["MonthlyPayment"].astype("category")
plots.dependence_cat(data_filtered_df[LABEL] , data_filtered_df["MonthlyPayment"])

In [ ]:
#create new category for missing
data_filtered_df["HomeOwnershipType"] = data_filtered_df["HomeOwnershipType"].cat.add_categories(["N/A"])
data_filtered_df["HomeOwnershipType"].fillna("N/A", inplace=True)
plots.dependence_cat(data_filtered_df[LABEL] , data_filtered_df["HomeOwnershipType"])

In [ ]:
#discretize, fill nan later with other features
data_filtered_df["NrOfDependants"] = data_filtered_df["NrOfDependants"].cat.add_categories(["3+"])
data_filtered_df["NrOfDependants"] = data_filtered_df["NrOfDependants"].replace(["10Plus", 10,9,8,7,6,5,4,3], "3+")
data_filtered_df["NrOfDependants"] = data_filtered_df["NrOfDependants"].cat.remove_unused_categories()
data_filtered_df["NrOfDependants"] = data_filtered_df["NrOfDependants"]
plots.dependence_cat(data_filtered_df[LABEL] , data_filtered_df["NrOfDependants"])

In [ ]:
#fill nans, median for continuous, mode for categoricals
filler = tran.NaN_filler()
data_df = filler.fit_transform(data_filtered_df)

In [ ]:
#check to be sure
utils.check_missing(data_df)

## Train-test split

In [ ]:
#stratified split before feature selection
train_df, test_df = utils.stratified_train_test_split(data_df, LABEL, test_size=0.3, random_state=SEED)

#### Export

In [ ]:
#export both sets before further analysis and selection
train_df.to_csv("train_full.csv", index=False)
test_df.to_csv("test_full.csv", index=False)

## Univariate analysis

#### Univariate scoring

In [ ]:
#sort predictors by in-sample univariate score (using logistic regression)
scores = utils.get_univariate_ginis(train_df, LABEL, model="logit", random_state=SEED)
scores.head(16)

In [ ]:
#work with selected features only
train_df = train_df[list(scores.head(15).index)+[LABEL]]
test_df = test_df[list(scores.head(15).index)+[LABEL]]

#### Coarse classification

In [ ]:
#Language code
#drop feature Country contains almost the same information in better format
plots.dependence_cat(train_df[LABEL] , train_df["LanguageCode"])
train_df.drop("LanguageCode", axis=1, inplace=True)

In [ ]:
#Country
#merge SK (too few observations) to ES
plots.dependence_cat(train_df[LABEL] , train_df["Country"])

country_map = {"SK" : "ES"}
train_df["Country"] = train_df["Country"].replace(country_map)
plots.dependence_cat(train_df[LABEL] , train_df["Country"])

In [ ]:
#Home ownership
plots.dependence_cat(train_df[LABEL] , train_df["HomeOwnershipType"])

#obviosly "overfitted"
def merge_home(row):
    if row in [7,8,9]:
        return "789"
    elif row in [2,3,4,0]:
        return "234"
    elif row in [5,6]:
        return "56"
    else:
        return row
train_df["HomeOwnershipType"] = train_df["HomeOwnershipType"].apply(merge_home)
train_df["HomeOwnershipType"] = train_df["HomeOwnershipType"].astype("category")
plots.dependence_cat(train_df[LABEL], train_df["HomeOwnershipType"])
score = utils.get_univariate_ginis(train_df[["HomeOwnershipType", LABEL]], LABEL, random_state=SEED)
score

In [ ]:
#Marital status
plots.dependence_cat(train_df[LABEL], train_df["MaritalStatus"])
#merge "divorced" (4) and "widowed" (5)
def merge_marital(row):
    if row in [4, 5]:
        return 45
    else:
        return row
train_df["MaritalStatus"] = train_df["MaritalStatus"].apply(merge_marital)
plots.dependence_cat(train_df[LABEL], train_df["MaritalStatus"])

In [ ]:
#OccupationArea
plots.dependence_cat(train_df[LABEL], train_df["OccupationArea"], figsize=(15,10))

In [ ]:
def merge_occupation(row):
    if row == 16:
        return "very low"
    if row in [3, 7, 10, 11, 13, 15, 18, 19]:
        return "low"
    if row in [1, 2, 4, 6, 8, 12, 14, 17]:
        return "medium"
    if row in [0, 5, 9]:
        return "high"
    else:
        return row

train_df["OccupationArea"] = train_df["OccupationArea"].apply(merge_occupation)
plots.dependence_cat(train_df[LABEL], train_df["OccupationArea"])
score = utils.get_univariate_ginis(train_df[["OccupationArea", LABEL]], LABEL, random_state=SEED)
score

In [ ]:
#UseOfLoan
#all 1xx to business (3) business logic
def merge_business(row):
    if row in [101, 102, 103, 104, 105, 106, 107, 108, 109, 110]:
        return 3
    else:
        return row
    
train_df["UseOfLoan"] = train_df["UseOfLoan"].apply(merge_business)
plots.dependence_cat(train_df[LABEL], train_df["UseOfLoan"], figsize=(15,10))

In [ ]:
def merge_use(row):
    if row in [4, 7, 8]:
        return "high"
    if row in [1 , 2, 3, 5, 6]:
        return "medium"
    if row == 0:
        return "low"
    else:
        return row
    
train_df["UseOfLoan"] = train_df["UseOfLoan"].apply(merge_use)
plots.dependence_cat(train_df[LABEL], train_df["UseOfLoan"])
score = utils.get_univariate_ginis(train_df[["UseOfLoan", LABEL]], LABEL, random_state=SEED)
score

#### Apply same transformations to test set

In [ ]:
test_df.drop("LanguageCode", axis=1, inplace=True)
test_df["Country"] = test_df["Country"].replace(country_map)
test_df["HomeOwnershipType"] = test_df["HomeOwnershipType"].apply(merge_home)
test_df["MaritalStatus"] = test_df["MaritalStatus"].apply(merge_marital)
test_df["OccupationArea"] = test_df["OccupationArea"].apply(merge_occupation)
test_df["UseOfLoan"] = test_df["UseOfLoan"].apply(merge_business)
test_df["UseOfLoan"] = test_df["UseOfLoan"].apply(merge_use)

#correct dtypes
feats = ["Country", "HomeOwnershipType", "MaritalStatus", "OccupationArea", "UseOfLoan"]
for feat in feats:
    train_df[feat] = train_df[feat].astype("category")
    train_df[feat] = train_df[feat].cat.remove_unused_categories()
    test_df[feat] = test_df[feat].astype("category")
    test_df[feat] = test_df[feat].cat.remove_unused_categories()

## Multivariate analysis

#### Pairwise correlations

In [ ]:
#woe-encode categoricals
woe = tran.WoE_transformer()
train_woe_df = woe.fit_transform(train_df, train_df[LABEL])

plots.plot_correlation_matrix(train_woe_df.drop(LABEL, axis=1), figsize=(20,20))

In [ ]:
#eliminate correlations >60% => NoOfPreviousLoansBeforeLoan, AmountOfPreviousLoansBeforeLoan, NewCreditCustomer
#keep NewCreditCustomer, simplest, ginis almost the same
train_df.drop("NoOfPreviousLoansBeforeLoan", axis=1, inplace=True)
test_df.drop("NoOfPreviousLoansBeforeLoan", axis=1, inplace=True)
train_df.drop("AmountOfPreviousLoansBeforeLoan", axis=1, inplace=True)
test_df.drop("AmountOfPreviousLoansBeforeLoan", axis=1, inplace=True)

#### Random forest relative feature importances

In [ ]:
#ohe-encode categoricals
ohe = tran.OHE_transformer()
X = ohe.fit_transform(train_df.drop(LABEL, axis=1))
y = train_df[LABEL]
#fit rf
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X.values, y.values)
#plot importances
plots.FeaturesImportanceTree(rf, X.columns, head=None, figsize=(11, 9))

## Export

In [ ]:
#export both sets after selection
train_df.to_csv("train_selected.csv", index=False)
test_df.to_csv("test_selected.csv", index=False)